Often I need to generate arguments or local variables through function calls, here is the parser

```
<SOURCE> := 'source :' ( <ATT_SOURCE> | <FUN_SOURCE> | <RAW> )
<RAW> := <str>|<int>|<float>
<ATT_SOURCE> := '{'
                '}'
<FUN_SOURCE> := '{' 
                'type: fun',
                'fun': str
                ('args:' <ARG_LIST>,)?
                ('kwargs:' <KWARG_LIST>,)?
                ('rename:' Dict[str, str],)?
                ('options:' (NoForward|)+)?
                ('store:' ('#':str | '$':str | str)? 
                '}' 
```

Reserved keywords are:
+ name
+ args
+ kwargs
+ rename
+ options
+ type

In [6]:
def function_arg_parser(source_parser, name, args=[], 
                        kwargs={}, rename=None, 
                        options=None, **other_confs):
    """
    Parses the function, returns a function that requires:
    + local namespace
    + *args
    + **kwargs
    
    And returns the value
    """
    fun = name
    args = [source_parser(i) for i in args]
    kwargs = {k: source_parser(v) for k, v in kwargs.items()}
    
    def return_fun(local, *n_args, **n_kwargs):
        eff_args = [i(local, *n_args, **n_kwargs) for i in args]
        eff_kwargs = {k: v(local, *n_args, **n_kwargs) for k,v in kwargs.items()}
        
        eff_args.extend(list(n_args))
        eff_kwargs.update(n_kwargs)
        
        globs = globals()
        f = eval(fun, globs, local)
        
        return f(*eff_args, **eff_kwargs)
    return return_fun

In [7]:
def attribute_arg_parser(source_parser, name, **other_confs):
    """
    Returns a function accepting a namespace, *args and **kwargs, ignores the ?args and 
    returns the value
    """
    def return_fun(local, *args, **kwargs):
        return eval(name, globals(), local)
    
    return return_fun

In [8]:
def kwarg_arg_parser(source_parser, name, dic_name="kwargs", **other_confs):
    def return_fun(local, *args, **kwargs):
        return eval(dic_name, globals(), local)[name]
    return return_fun

In [9]:
def source_parse(configuration):
    if type(configuration) != dict:
        def simple_res(*args, **kwargs):
            return configuration
        return simple_res
    
    t = configuration['type']
    if t == 'fun':
        parsed = function_arg_parser(source_parse, **configuration)
    elif t == 'att':
        parsed = attribute_arg_parser(source_parse, **configuration)
    elif t == 'kwarg':
        parsed = kwarg_arg_parser(source_parse, **configuration)
    else: 
        raise AttributeError('Neither function nor attribute')
    
    store_action = configuration.get('store', None)
    rename = configuration.get('rename', None)
    options = configuration.get('options', [])
    
    def function_returned(local, *args, **kwargs):
        if 'NoForward' in options:
            args = []
            kwargs = {}
        r = parsed(local, *args, **kwargs)
        
        if rename is not None:
            r = r.rename(columns=rename)
        if store_action is None:
            pass
        elif store_action[0] == '#':
            setattr(locs['self'], store_action[1:], r)
        elif store_action[0] == '$':
            d, k = tuple(store_action[1:].split('.')[:2])
            local[d][k]=r
        else:
            locs[store_action]=r
        return r
    
    return function_returned
    

In [10]:
def test(*args, **kwargs):
    print(args, kwargs)

def test_vuoto():
    print("test vuoto")
    return 20
    
ciao = 1

conf = {'args': [10, {'name': 'ciao', 'type': 'att'}],
 'kwargs': {'a': {'name': 'test_vuoto', 'options': 'NoForward', 'type': 'fun'}},
 'name': 'test',
 'type': 'fun'}

In [11]:
f = source_parse(conf)

In [12]:
f(locals(), 2, "ciao", b=20, a=2)

test vuoto
(10, 1, 2, 'ciao') {'a': 2, 'b': 20}
